<a href="https://colab.research.google.com/github/Trading-com-Dados/desenvolvimento_interno/blob/main/001up_20220402_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **SCREENING: varrendo uma lista de ativos em poucos segundos** 
---
#### *Disclaimer:*
#### Este código foi elaborado para fins exclusivamente educacionais. O conteúdo aqui apresentado visa fornecer informações complementares para auxiliar o investidor na tomada de suas próprias decisões de investimento. Reforçando, nenhum tópico aqui abordado constitui qualquer tipo de indicação/oferta/solicitação de compra/venda de qualquer produto.
---



# **1. Bibliotecas utilizadas**

In [ ]:
!pip install pandas numpy
!pip install matplotlib plotly
!pip install yfinance
!pip install ta
!pip install -q kora

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
from time import sleep
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import ta as ta
from kora.selenium import wd
from google.colab.data_table import DataTable

%matplotlib inline

# **2. Obtendo cotações ao vivo dos ativos**

## 2.1 Lista de ativos do site B3 por WebScraping

#### https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.html
#### https://www.b3.com.br/pt_br/market-data-e-indices/
#### https://www.youtube.com/watch?v=b9bU_ryC5rc
#### https://www.selenium.dev/downloads/
#### https://github.com/korakot/kora
#### https://www.youtube.com/watch?v=b9bU_ryC5rc

In [ ]:
def composicao_carteira_B3(indice,espera=8):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  wd.get(url)
  sleep(espera)

  wd.find_element_by_id('segment').send_keys("Setor de Atuação")
  sleep(espera)

  wd.find_element_by_link_text("Download").click()
  sleep(espera)

  arquivos = !ls -1t *.csv
  return pd.read_csv(arquivos[0],sep=';',encoding='ISO-8859-1',skipfooter=2,engine='python',thousands='.',decimal=',',header=1,index_col=False)

In [ ]:
## Pode utilizar qualquer índice da B3 (IBrA, IBrX)
ativos_ibov = composicao_carteira_B3('ibov')

In [ ]:
ativos_ibov

In [ ]:
tickers_ibov = ativos_ibov['Código']
tickers_ibov

In [ ]:
tickers_ibov_ls = list(tickers_ibov)
tickers_ibov_ls.sort()
tickers_ibov_ls

In [ ]:
ativo_SA = 0
tickers_ibov_ls_SA = []

for i in tickers_ibov_ls:
  ativo_SA = (i+'.SA')
  tickers_ibov_ls_SA.append(ativo_SA)

In [ ]:
## É interessante que a lista de ativos seja organizada de acordo com as próprias demandas
ativo_SA
tickers_ibov = tickers_ibov_ls_SA
tickers_ibov

## 2.2 Baixar as cotações até o momento atual

In [ ]:
## Defino a data atual
hoje = datetime.today()
hoje = hoje.strftime('%Y-%m-%d')
hoje

'2022-02-04'

In [ ]:
cotacoes_B3 = yf.download(tickers_ibov, start='2021-01-01', end=hoje, period = "1d")
cotacoes_B3

## Manipulando dataframes com NAs e NaN

In [ ]:
## Nos próximos passos estão alguns tipos de manipulações de NA e NaN
count_null = ativos_Adj_Close.isna().sum()
count_null

In [ ]:
np.median(count_null)
plt.boxplot(count_null);

In [ ]:
ativos_Adj_Close[(ativos_Adj_Close.isna().sum(1)/ativos_Adj_Close.shape[1])>0.9]

In [ ]:
ativos_Adj_Close_filtered = ativos_Adj_Close.drop(ativos_Adj_Close.columns[ativos_Adj_Close.isna().sum()>(len(ativos_Adj_Close.columns)*0.005)],axis = 1)
ativos_Adj_Close_filtered

In [ ]:
count_null = ativos_Adj_Close_filtered.isna().sum()
plt.boxplot(count_null);

In [ ]:
ativos_Adj_Close_filtered[(ativos_Adj_Close_filtered.isna().sum(1))>1]

In [ ]:
ativos_Adj_Close_filtered.dropna(subset = ["AALR3.SA"], inplace=True)

In [ ]:
ativos_Adj_Close_filtered

## Simplificar o número de colunas - "Wide to Long" 

In [ ]:
ativos_close = ativos_Adj_Close_filtered
ativos_close

In [ ]:
ativos_close['Data'] = ativos_close.index

In [ ]:
ativos_close = pd.melt(ativos_close,id_vars=['Data'],var_name='Ativos', value_name='Adj_Close')
ativos_close

# Cálculo Indicadores

In [ ]:
ativos_close2 = ativos_close

In [ ]:
sma_short = ta.trend.SMAIndicator(ativos_close2['Adj_Close'], window=8)
sma_medium = ta.trend.SMAIndicator(ativos_close2['Adj_Close'], window=20)
sma_long = ta.trend.SMAIndicator(ativos_close2['Adj_Close'], window=200)

In [ ]:
ativos_close2['SMA_SHORT'] = sma_short.sma_indicator()
ativos_close2['SMA_MEDIUM'] = sma_medium.sma_indicator()
ativos_close2['SMA_LONG'] = sma_long.sma_indicator()

In [ ]:
ativos_close2.head(50)

In [ ]:
rsi_14 = ta.momentum.RSIIndicator(close=ativos_close2['Adj_Close'], window=14)
ativos_close2['RSI_14'] = rsi_14.rsi()

In [ ]:
ativos_close2.head()

In [ ]:
import numpy as np
ts = pd.DataFrame()
ativos_close2['entrada'] = np.where(ativos_close2['Adj_Close']>ativos_close2['SMA_SHORT'],1,0)
ativos_close2

In [ ]:
ativos_close2

In [ ]:
df.loc[df.index > pd.to_datetime(order['transactionTime'], unit='ms')]

# Screening Ativos

In [ ]:
ativos_close2

In [ ]:
ativos_close_ontem = ativos_close2['Data'] == '2022-01-24'
ativos_close_ontem

In [ ]:
ativos_close_ontem2 = ativos_close2[ativos_close_ontem]
ativos_close_ontem2

In [ ]:
ativos_close_ontem2['MM_alinhadas']=ativos_close_ontem2['Adj_Close']>ativos_close_ontem2["SMA_SHORT"]

In [ ]:
len(ativos_close_ontem2['Adj_Close'])

In [ ]:
datas_uteis = ['2022-01-21','2022-01-24']

In [ ]:
ativos_close_ult_2 = ativos_close2.loc[ativos_close2['Data'].isin(datas_uteis)]
ativos_close_ult_2

In [ ]:
for i in range(0:1000):
    
    print(i)

In [ ]:
if ativos_close_ontem2['Adj_Close'] > ativos_close_ontem2['SMA_SHORT']:
    if ativos_close_ontem2['Adj_Close'] > ativos_close_ontem2['SMA_MEDIUM']:
        if ativos_close_ontem2['Adj_Close'] > ativos_close_ontem2['SMA_MEDIUM']:
            print(ativos_close_ontem2['Ativos'])
        else:
    else:
else:
    print("FALSE")

In [ ]:
datas_uteis = ['2022-01-21','2022-01-24']

In [ ]:
xxx = ativos_close.loc[ativos_close['Data'].isin(datas_uteis)]
xxx

In [ ]:
ativos_close_ontem2 = ativos_close[ativos_close_ontem]
ativos_close_ontem2